In [ ]:
import requests
import json
import time
from IPython.display import HTML  

In [ ]:
api_keys=["sk-1"，"sk-2"]

In [ ]:
url = "https://api.lixining.com/v1/chat/completions" # 选择一个API
#反向代理 如失效需要 更换到上面
#https://openai.1rmb.tk/v1/chat/completions
#https://ai.fakeopen.com/v1/chat/completions
#https://api.lixining.com/v1/chat/completions

In [ ]:
class APIPool:
    def __init__(self,tokens):
        self.tokens=tokens
        self.last_called={token:0 for token in tokens}  #上次调用时间
    def getapi(self):
        now = int(time.time())
        for token in self.tokens:
            if self.last_called[token]+21 <=now:    
                self.last_called[token]=now
                return token
        time.sleep(self.last_called[self.tokens[0]]+21-now) #等待
        return self.getapi() #重新调用
    def reset(self):
        self.last_called={token:0 for token in self.tokens}#重置

In [ ]:
A=APIPool(api_keys) #创建API池

In [ ]:
def get_header():# 获取请求头
    header={
  'Authorization': f'Bearer {A.getapi()}',
  'Content-Type': 'application/json'
    }
    return header

In [ ]:
#获取消息的函数
def get_completion(prompt):
    payload = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "system", "content": f"You are ChatGPT, a large language model trained by OpenAI. \
            Follow the user's instructions carefully. Respond using markdown."},
        {
          "role": "user",
          "content": f"{prompt}"
        }
      ]
    }
    data=None
    try:
        response = requests.post(url, headers=get_header(), json=payload)
        response.raise_for_status() # 抛出异常，如果响应码不是200
        data = response.json()
        return (data["choices"][0]["message"]["content"])
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {e} \n {data}")
    except json.JSONDecodeError as e:
        print(f"无效的 JSON 响应: {e}\n {data}")
    except KeyError as e:
        print(f"解析 JSON 错误: {e}\n {data}")

In [ ]:
class TextStorage:
    def __init__(self):
        self.text = ""

    def add_text(self, new_text):
        self.text += str(new_text)

    def get_text(self):
        return self.text
    
    def clear_text(self):
        self.text = ""
T=TextStorage()

In [ ]:
with open("./data/readinglist.json", "r") as f:
    reading_list = json.load(f)

In [ ]:
reading_list

In [ ]:
prompt = f"""
下面是三个引号括起来的JSON格式数据，增加一个列名为简介的列，把简介补充好,转换成HTML表格,表格要有分割线样式，只回答我html代码:

"""
for book in reading_list: 
    display((json.dumps(book["书名"],ensure_ascii=False)))
    response = get_completion(f"{prompt}+\"\"\"{json.dumps(book)}\"\"\"")
    T.add_text(response)
    

In [31]:
HTML(T.get_text())

书名,作者,出版年份,简介
计算机程序设计艺术,Donald E. Knuth,1968-1974,以深入讲解计算机程序设计的艺术与科学的方法著称，是计算机领域的名著之一。
书名,作者,出版年份,简介
人月神话,Fred Brooks,1975,本书主要阐述了软件开发管理中的一些原则和问题，尤其注重人员配备和沟通协作的重要性。
书名,作者,出版年份,简介
操作系统概念,"Silberschatz, Galvin 和 Gagne",1983,该书是操作系统领域的经典教材，全面覆盖操作系统的基本概念、设计原则和实现技术等方面的内容。
书名,作者,出版年份,简介
编辑原理,Alfred V. Aho、Monica S. Lam、Ravi Sethi和Jeffrey D. Ullman,1986,一本讲解编译原理和编译器构造的经典教材。
书名,作者,出版年份,简介
Unix 编程艺术,Eric Steven Raymond,1991,介绍 Unix 操作系统的特点和设计哲学以及 UNIX 命令行的常用工具和技巧
书名,作者,出版年份,简介
